## Семинар 5
# Тема: Отбор наилучшей модели из нескольких с наилучшими гиперпараметрами поиском по сетке

   Рассмотрим следующую задачу: необходимо подобрать наилучшую модель (из нескольких) с наилучшими гиперпараметрами, а затем для выбранной модели оценить качество обобщающей способности. Это можно сделать с помощью поиска по сетке, в которой присутствует и вид модели тоже (а не только значения гиперпараметров модели) с перекрёстной проверкой.

Импортируем необходимые библиотеки:

In [17]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris, load_breast_cancer
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV

### 1. Загрузите встроенный датасет load_breast_cancer. Обозначьте признаки за X, а целевую переменную за y и выведите их размеры. Выведите данные в виде датафрейма.

In [18]:
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target

X.shape, y.shape

((569, 30), (569,))

In [19]:
data = pd.DataFrame(X, columns=cancer.feature_names)
data['target'] = y
data.head(5)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [20]:
data['target'].value_counts()

target
1    357
0    212
Name: count, dtype: int64

In [21]:
print(cancer.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

:Number of Instances: 569

:Number of Attributes: 30 numeric, predictive attributes and the class

:Attribute Information:
    - radius (mean of distances from center to points on the perimeter)
    - texture (standard deviation of gray-scale values)
    - perimeter
    - area
    - smoothness (local variation in radius lengths)
    - compactness (perimeter^2 / area - 1.0)
    - concavity (severity of concave portions of the contour)
    - concave points (number of concave portions of the contour)
    - symmetry
    - fractal dimension ("coastline approximation" - 1)

    The mean, standard error, and "worst" or largest (mean of the three
    worst/largest values) of these features were computed for each image,
    resulting in 30 features.  For instance, field 0 is Mean Radius, field
    10 is Radius SE, field 20 is Worst Radius.

    - 

### 2. Разбейте данные на два набора обучающий и тестовый при помощи train_test_split, зафиксировав random_state=0. Выведите размеры полученных наборов.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
x_train.shape, x_test.shape

((455, 30), (114, 30))

### 3. Используя GridSearchCV, подберите наилучшую модель классификатора на обучающем наборе данных, выбирая между методом опорных векторов и деревом решений, подобрав наилучшие гиперпараметры для этих моделей, используя следующие сетки гиперпараметров. Выведите получившуюся модель с наилучшими значениями гиперпараметров.

In [23]:
param_svc= {'classifier': [SVC()], 
            'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]}
param_tree = {'classifier': [DecisionTreeClassifier(random_state = 42)],
              "classifier__max_depth": [1, 3, 5, 7, 10],
              "classifier__min_samples_leaf": [1, 3, 5, 10],
              "classifier__criterion": ["gini", "entropy"]}

In [44]:
param_grid = [param_svc, param_tree]
pipe = Pipeline([('classifier', SVC())])
pipe.get_params()

{'memory': None,
 'steps': [('classifier', SVC())],
 'transform_input': None,
 'verbose': False,
 'classifier': SVC(),
 'classifier__C': 1.0,
 'classifier__break_ties': False,
 'classifier__cache_size': 200,
 'classifier__class_weight': None,
 'classifier__coef0': 0.0,
 'classifier__decision_function_shape': 'ovr',
 'classifier__degree': 3,
 'classifier__gamma': 'scale',
 'classifier__kernel': 'rbf',
 'classifier__max_iter': -1,
 'classifier__probability': False,
 'classifier__random_state': None,
 'classifier__shrinking': True,
 'classifier__tol': 0.001,
 'classifier__verbose': False}

In [26]:
gs = GridSearchCV(estimator=pipe,
                  param_grid=param_grid,
                  cv = 5)

In [28]:
gs.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=Pipeline(steps=[('classifier', SVC())]),
             param_grid=[{'classifier': [SVC()],
                          'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]},
                         {'classifier': [DecisionTreeClassifier(random_state=42)],
                          'classifier__criterion': ['gini', 'entropy'],
                          'classifier__max_depth': [1, 3, 5, 7, 10],
                          'classifier__min_samples_leaf': [1, 3, 5, 10]}])

In [58]:
pd.DataFrame(gs.cv_results_)[['params', 'mean_test_score']]

,params,mean_test_score
0,"{'classifier': SVC(), 'classifier__C': 0.001}",0.637363
1,"{'classifier': SVC(), 'classifier__C': 0.01}",0.701099
2,"{'classifier': SVC(), 'classifier__C': 0.1}",0.887912
3,"{'classifier': SVC(), 'classifier__C': 1}",0.905495
4,"{'classifier': SVC(), 'classifier__C': 10}",0.914286
5,"{'classifier': SVC(), 'classifier__C': 100}",0.931868
6,{'classifier': DecisionTreeClassifier(random_s...,0.892308
7,{'classifier': DecisionTreeClassifier(random_s...,0.892308
8,{'classifier': DecisionTreeClassifier(random_s...,0.892308
9,{'classifier': DecisionTreeClassifier(random_s...,0.892308


In [37]:
gs.best_params_

{'classifier': DecisionTreeClassifier(random_state=42),
 'classifier__criterion': 'entropy',
 'classifier__max_depth': 7,
 'classifier__min_samples_leaf': 3}

In [39]:
gs.best_score_

np.float64(0.9340659340659341)

### 4. Оцените качество наилучшей модели, выведя значения метрик accuracy и f1-score на тестовых данных.

In [38]:
gs.score(x_test, y_test)

0.9298245614035088

In [42]:
y_test_predict = gs.predict(x_test)
accuracy = gs.score(x_test, y_test)
accuracy

0.9298245614035088

In [43]:
f1 = metrics.f1_score(y_test, y_test_predict)
f1

0.9384615384615385

### 5. Добавьте масштабирование данных методом min-max в метод опорных векторов и снова подберите наилучшую модель классификатора, выбирая между методом опорных векторов и деревом решений с теми же диапазонами гиперпараметров. Выведите получившуюся модель с наилучшими значениями гиперпараметров.

In [45]:
param_svc_scaler= {'classifier': [SVC()], 'preprocessing':[MinMaxScaler()],
            'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]}
param_tree_scaler = {'classifier': [DecisionTreeClassifier(random_state = 42)], 'preprocessing':[None],
              "classifier__max_depth": [1, 3, 5, 7, 10],
              "classifier__min_samples_leaf": [1, 3, 5, 10],
              "classifier__criterion": ["gini", "entropy"]}

param_grid_scaler = [param_svc_scaler, param_tree_scaler]

In [46]:
pipe_scaler = Pipeline([('preprocessing', MinMaxScaler()), ('classifier', SVC())])

In [47]:
gs_scaler = GridSearchCV(estimator = pipe_scaler,
                       param_grid = param_grid_scaler,
                       cv = 5)

In [51]:
gs_scaler.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing', MinMaxScaler()),
                                       ('classifier', SVC())]),
             param_grid=[{'classifier': [SVC()],
                          'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
                          'preprocessing': [MinMaxScaler()]},
                         {'classifier': [DecisionTreeClassifier(random_state=42)],
                          'classifier__criterion': ['gini', 'entropy'],
                          'classifier__max_depth': [1, 3, 5, 7, 10],
                          'classifier__min_samples_leaf': [1, 3, 5, 10],
                          'preprocessing': [None]}])

In [57]:
pd.DataFrame(gs_scaler.cv_results_)[['param_classifier', 'param_preprocessing', 'mean_test_score']]

,param_classifier,param_preprocessing,mean_test_score
0,SVC(),MinMaxScaler(),0.637363
1,SVC(),MinMaxScaler(),0.637363
2,SVC(),MinMaxScaler(),0.949451
3,SVC(),MinMaxScaler(),0.980220
4,SVC(),MinMaxScaler(),0.973626
5,SVC(),MinMaxScaler(),0.953846
6,DecisionTreeClassifier(random_state=42),None,0.892308
7,DecisionTreeClassifier(random_state=42),None,0.892308
8,DecisionTreeClassifier(random_state=42),None,0.892308
9,DecisionTreeClassifier(random_state=42),None,0.892308


In [59]:
gs_scaler.best_params_

{'classifier': SVC(), 'classifier__C': 1, 'preprocessing': MinMaxScaler()}

### 6. Оцените качество полученной наилучшей модели с учётом масштабирования данных, выведя значения метрик accuracy и f1-score на тестовых данных. Сделайте вывод о том нужно ли было делать масштабирование данных.

In [61]:
gs_scaler.score(x_test, y_test), accuracy

(0.9736842105263158, 0.9298245614035088)

In [64]:
y_test_predict_scaler = gs_scaler.predict(x_test)
metrics.f1_score(y_test, y_test_predict_scaler), f1

(0.9777777777777777, 0.9384615384615385)

## Задания для самостоятельного выполнения
1. Загрузите встроенный датасет `load_iris`. Обозначьте данные за `X`, а целевую переменную за `y`.
2. Разбейте данные на два набора: обучающий и тестовый, взяв в тестовый набор 20% данных и указав random_state=0.
3. Используя `GridSearchCV`, подберите наилучшую модель классификатора на обучающем наборе данных, выбирая между методом логистической регрессии и деревом со следующими наборами гиперпараметров: param_grid = [{'classifier': [LogisticRegression(max_iter=500)],'classifier__C': np.logspace(0,4,10), 'classifier__penalty': ['l2', 'l1'], 'classifier__solver': ['liblinear']}, {'classifier': [LogisticRegression(max_iter=500)],'classifier__C': np.logspace(0,4,10), 'classifier__penalty': ['l2'], 'classifier__solver': ['lbfgs', 'newton-cg']}, {'classifier': [DecisionTreeClassifier(random_state = 42)], 'classifier__max_depth': [1, 3, 5, 7], 'classifier__criterion': ["gini", "entropy"], 'classifier__max_features': [1, 2, 3]}]. Выведите получившуюся модель с наилучшими значениями гиперпараметров.
4. Оцените качество наилучшей модели, выведя значения метрик accuracy и f1-score на тестовых данных.
5. Добавьте стандартную нормализацию данных в метод логистической регрессии и снова подберите наилучшую модель классификатора, выбирая между методом логистической регрессии и деревом решений с теми же диапазонами гиперпараметров. Выведите получившуюся модель с наилучшими значениями гиперпараметров.
6. Оцените качество полученной наилучшей модели с учётом стандартной нормализации данных, выведя значения метрик accuracy и f1-score на тестовых данных. Сделайте вывод о том нужна ли здесь была нормализация данных.
7. Добавить в param_grid 3 модель k-ближайший соседей
8. Найти лучшую модель с RandomGridSearch

In [69]:
iris = load_iris()
X = iris.data
y = iris.target

X.shape, y.shape

((150, 4), (150,))

In [71]:
data = pd.DataFrame(X, columns=iris.feature_names)
data['target'] = y
data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train.shape, X_test.shape

((120, 4), (30, 4))

In [84]:
param_grid = [{'classifier': [LogisticRegression(max_iter=500)],
'classifier__C': np.logspace(0,4,10), 'classifier__penalty': ['l2', 'l1'], 'classifier__solver': ['liblinear']},
{'classifier': [LogisticRegression(max_iter=500)],'classifier__C': np.logspace(0,4,10), 'classifier__penalty': ['l2'], 
'classifier__solver': ['lbfgs', 'newton-cg']}, {'classifier': [
DecisionTreeClassifier(random_state = 42)], 'classifier__max_depth': [1, 3, 5, 7],
'classifier__criterion': ["gini", "entropy"], 'classifier__max_features': [1, 2, 3]}]

pipe = Pipeline([('classifier', LogisticRegression(max_iter=500))])
pipe.get_params()

{'memory': None,
 'steps': [('classifier', LogisticRegression(max_iter=500))],
 'transform_input': None,
 'verbose': False,
 'classifier': LogisticRegression(max_iter=500),
 'classifier__C': 1.0,
 'classifier__class_weight': None,
 'classifier__dual': False,
 'classifier__fit_intercept': True,
 'classifier__intercept_scaling': 1,
 'classifier__l1_ratio': None,
 'classifier__max_iter': 500,
 'classifier__multi_class': 'deprecated',
 'classifier__n_jobs': None,
 'classifier__penalty': 'l2',
 'classifier__random_state': None,
 'classifier__solver': 'lbfgs',
 'classifier__tol': 0.0001,
 'classifier__verbose': 0,
 'classifier__warm_start': False}

In [85]:
gs = GridSearchCV(estimator=pipe,
                  param_grid=param_grid,
                  cv = 5)

In [86]:
gs.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('classifier',
                                        LogisticRegression(max_iter=500))]),
             param_grid=[{'classifier': [LogisticRegression(max_iter=500)],
                          'classifier__C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04]),
                          'classifier__penalty...
                          'classifier__C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04]),
                          'classifier__penalty': ['l2'],
                          'classifier__solver': ['lbfgs', 'newton-cg']},
                         {'classifier': [DecisionTreeClassifier(random_state=42)],
                          'classifier__criterion': ['gini', 'entropy'],
                          'classifier__max_depth': [1, 3, 5, 7],
                          'classifier__max_features': [1, 2, 3]}])

In [87]:
gs.best_params_

{'classifier': LogisticRegression(max_iter=500),
 'classifier__C': np.float64(21.544346900318832),
 'classifier__penalty': 'l1',
 'classifier__solver': 'liblinear'}

In [90]:
accuracy = gs.score(X_test, y_test)

In [91]:
accuracy

1.0

In [95]:
y_test_predict = gs.predict(X_test)
f1 = metrics.f1_score(y_test, y_test_predict)
f1

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].